In [1]:
%load_ext autoreload
%autoreload 2
"""
    Some imports used throughout the notebook
"""
import time

from visualize import *

from cpmpy.transformations.normalize import toplevel_list
from factory import *
from read_data import get_data
from IPython.display import clear_output


import numpy as np
np.set_printoptions(linewidth=90)
# preload solvers
from cpmpy import SolverLookup
names = SolverLookup.solvernames()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-03


## Hands-on causal explanations

In this notebook, you will get experience with computing causal explanations.

We focus on explaining unsatisfiability by means of extracting a MUS.



<img src="img/mus.png" width="20%" align="right" style="margin:50px;">

Trim model to minimal set of constraints

... minimize cognitive burden for user

In [2]:
instance = "Benchmarks/Instance2.txt"
data = get_data(instance)
factory = NurseSchedulingFactory(data)

Let's have a look at the instance and its (optimal) solution.

In [3]:
data.staff[["name", "MaxShifts","MaxWeekends"]]

,name,MaxShifts,MaxWeekends
0,Megan,E=14|L=14,1
1,Katherine,E=14|L=14,1
2,Robert,E=14|L=14,1
3,Jonathan,E=14|L=0,1
4,William,E=0|L=14,1
5,Richard,E=14|L=14,1
6,Kristen,E=14|L=14,1
7,Kevin,E=14|L=14,1
8,Thomas,E=14|L=14,1
9,Brandy,E=14|L=14,1


In [4]:
print(f"Planning for {data.horizon} days")

Planning for 14 days


In [5]:
import random

In [6]:
model, nurse_view = factory.get_optimization_model()
random.shuffle(model.constraints)
assert model.solve(solver="ortools") # you can try different solvers here!


print(model.status())
print("Total penalty:", model.objective_value())
visualize(nurse_view.value(), factory)

ExitStatus.OPTIMAL (1.859007 seconds)
Total penalty: 828


In [7]:
requests, _ = factory.shift_on_requests(formulation="hard")

denied_requests = [req for req in requests if req.value() is False]
print("The following requests were denied:")
for req in denied_requests:
    print("-", req)

visualize_constraints(denied_requests, nurse_view, factory, do_clear=False)

The following requests were denied:
- Katherine requests to work shift E on Thu 2
- Robert requests to work shift E on Tue 2
- William requests to work shift L on Sat 2
- William requests to work shift L on Sun 2
- Rebecca requests to work shift L on Mon 2
- Rebecca requests to work shift L on Wed 2
- Juan requests to work shift L on Thu 1
- Juan requests to work shift L on Fri 1
- Juan requests to work shift L on Fri 2
- Katelyn requests to work shift L on Thu 1
- Katelyn requests to work shift L on Fri 1
- Katelyn requests to work shift L on Mon 2
- Christine requests to work shift E on Mon 1
- Christine requests to work shift E on Tue 1
- Christine requests to work shift E on Wed 2
- Christine requests to work shift E on Thu 2


In [8]:
# try it yourself!

# requests, _ = factory.shift_off_requests(formulation="hard")
# cover_constraints, _ = factory.cover(formulation="hard")

# TODO: find out which are not satisfied, and visualize!

## Deductive explanations

In the remainder of this notebook, we will explore different ways of explaining unsatisfiabily of the instance.

In [9]:
model, nurse_view = factory.get_decision_model()
model.solve()

False

In [10]:
from cpmpy.tools.explain import mus

t0 = time.time()
conflict = mus(model.constraints) # try different solvers here!
print(f"Found conflict of size {len(conflict)} in {round(time.time()-t0,2)}s")

Found conflict of size 3 in 1.04s


In [11]:
for c in conflict:
    print("-", c)

visualize_constraints(conflict, nurse_view, factory)

- William should work at most 1 weekends
- William requests to work shift L on Sat 1
- William requests to work shift L on Sat 2


Now, let's influence the MUS we would like to find.

We can chose from QuickXplain [1] or Optimal MUS (OUS) [2]

**QuickXplain** takes as input a total ordering of constraints, and returns a lexicographically minimal MUS.
The algorithm is build up as a divide-and-conquer approach, and therefore has a good average complexity.

**OUS** takes as input a weight for each constraint, and finds a **optimal** MUS. While this optimality guarantee is sometimes required, it comes at a penalty of longer computation times, as you will notice here!

In [12]:
# QuickXplain first
from cpmpy.tools.explain import quickxplain


def get_weight(cons):
    if "William" in str(cons): # Find a different MUS than the previous
        return 2 
    return 1

ordered = sorted(model.constraints, key=get_weight)
conflict = quickxplain(ordered)
print(f"Found conflict of size {len(conflict)} in {round(time.time()-t0,2)}s")

Found conflict of size 6 in 5.12s


In [13]:
for c in conflict:
    print("-", c)

visualize_constraints(conflict, nurse_view, factory)

- Juan cannot work more than 2160min
- Juan requests to work shift L on Thu 1
- Juan requests to work shift L on Fri 1
- Juan requests to work shift L on Thu 2
- Juan requests to work shift L on Fri 2
- Juan requests to work shift L on Sat 2


In [14]:
# Now find truely OPTIMAL MUSes
## Careful, this takes a while if you are not using Exact!
from cpmpy.tools.explain import optimal_mus

def get_weight(cons):
    if "william" in str(cons).lower():
        return 5
    else:
        return 1
#     return len(str(cons)) # favor constraints with short description


solver = "exact" if "exact" in cp.SolverLookup.solvernames() else "ortools"
print("Using solver", solver)

conflict = optimal_mus(model.constraints, 
                       weights=[get_weight(c) for c in model.constraints],
                       solver=solver,
                       hs_solver="gurobi")
print(f"Found conflict of size {len(conflict)} in {round(time.time()-t0,2)}s")

Using solver exact
Found conflict of size 4 in 14.56s


In [15]:
for c in conflict:
    print("-", c)

visualize_constraints(conflict, nurse_view, factory)

- Megan requests to work shift L on Mon 2
- Rebecca requests to work shift L on Mon 2
- Katelyn requests to work shift L on Mon 2
- Shift L on Mon 2 must be covered by 2 nurses out of 14
